In [16]:
import pandas as pd

In [17]:
df = pd.read_table("./Data/validated.tsv")
df.head()

,client_id,path,sentence,up_votes,down_votes,age,gender,accents,variant,locale,segment
0,00778b9356111e5c05365103769fb9ec90fc07e08476ad...,common_voice_en_37476269.mp3,This led to both the Red Terror and the White ...,2,0,NaN,NaN,United States English,NaN,en,NaN
1,023a5e6a4ca85bad19139255b57274785ae3faae96a789...,common_voice_en_37406282.mp3,Its members include representatives of the tea...,2,0,NaN,NaN,NaN,NaN,en,NaN
2,02efbf700a190d3114277b0b5e1a0cb00f653cfcd215ce...,common_voice_en_37740889.mp3,Paper wine labels have long been collected.,2,0,NaN,NaN,England English,NaN,en,NaN
3,033de9ff626035f924f0c3e5a907c459b0a5f28c4bdc1c...,common_voice_en_37355311.mp3,I wondered if those birds were redwings?,2,0,NaN,NaN,NaN,NaN,en,NaN
4,05627b7dd6431caab6b11370280f496f99805ce901f0b5...,common_voice_en_37442699.mp3,Henry Heimlich also promoted abdominal thrusts...,2,0,NaN,NaN,NaN,NaN,en,NaN


In [18]:
# keep only rows of top 10 accents
top10 = df['accents'].value_counts().head(10).index
df = df[df['accents'].isin(top10)]
df['accents'].value_counts()

accents
United States English                                 960
Southern African (South Africa, Zimbabwe, Namibia)    884
Canadian English                                      348
German English,Non native speaker                     346
United States English,Scandinavian                    332
England English                                       173
Malaysian English                                      93
Australian English                                     59
United States English,Filipino                         52
United States English,Like a Kennedy ,Boston           52
Name: count, dtype: int64

In [19]:
df = df[['path', 'accents', 'sentence']]
df.head()


,path,accents,sentence
0,common_voice_en_37476269.mp3,United States English,This led to both the Red Terror and the White ...
2,common_voice_en_37740889.mp3,England English,Paper wine labels have long been collected.
6,common_voice_en_37607972.mp3,United States English,The number of chime sets matches the number of...
8,common_voice_en_37437567.mp3,United States English,The Atbara flows only while there is rain in E...
10,common_voice_en_37288403.mp3,United States English,Katsande and her son were accused of voter int...


In [20]:
df.shape

(3299, 3)

In [21]:
df.to_csv('Data/validated_top10.csv', index=False)

In [22]:
df = pd.read_csv('Data/validated_top10.csv', index_col=False)
df.head()

,path,accents,sentence
0,common_voice_en_37476269.mp3,United States English,This led to both the Red Terror and the White ...
1,common_voice_en_37740889.mp3,England English,Paper wine labels have long been collected.
2,common_voice_en_37607972.mp3,United States English,The number of chime sets matches the number of...
3,common_voice_en_37437567.mp3,United States English,The Atbara flows only while there is rain in E...
4,common_voice_en_37288403.mp3,United States English,Katsande and her son were accused of voter int...


In [23]:
end = round(0.9 * df.shape[0])
train_lines = df[:end]
test_lines = df[end:]

In [24]:
import librosa

In [25]:
import numpy as np
audio_path = './Data/clips/common_voice_en_37288403.mp3'
y, sr = librosa.load(audio_path)

# Compute MFCCs
mfcc = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=128).T, axis= 0)

In [26]:
y.shape

(106370,)

In [27]:
import sys
sys.path.append('rnnt/')
from AudioDataset import AudioDataset

In [28]:
end = round(0.9 * df.shape[0])
train = df[:end]
test = df[end:]

train_set = AudioDataset(train['path'], train['sentence'], train['accents'])
test_set = AudioDataset(test['path'], test['sentence'], test['accents'])

In [29]:
import tqdm


pbar = tqdm(train_set.loader)

AttributeError: 'AudioDataset' object has no attribute 'loader'

In [ ]:
mfccs = train_set.__getitem__(0)[0]

In [ ]:
mfccs

array([[-6.4883484e+02, -6.4883484e+02, -6.4849475e+02, ...,
        -5.2956940e+02, -5.2614172e+02, -5.4585895e+02],
       [ 0.0000000e+00,  0.0000000e+00, -2.3577055e-01, ...,
        -2.5695023e+01, -2.7831463e+01, -2.0191107e+01],
       [ 0.0000000e+00,  0.0000000e+00, -2.4970216e-01, ...,
        -1.0212838e+01, -1.5875332e+01, -1.7488293e+01],
       ...,
       [ 0.0000000e+00,  0.0000000e+00, -1.9037853e-01, ...,
        -1.4437845e+01, -1.2097955e+01, -8.4394569e+00],
       [ 0.0000000e+00,  0.0000000e+00, -2.8653240e-01, ...,
        -6.0396314e-02,  5.8689275e+00,  4.1454673e+00],
       [ 0.0000000e+00,  0.0000000e+00,  4.6985769e-01, ...,
         6.2060242e+00,  6.7223449e+00,  2.7742450e+00]], dtype=float32)